In [1]:
from __future__ import division
import numpy as np
from matplotlib import pyplot as plt
import glob, operator, time, shutil, scipy, sys
import pandas as pd

pd.set_option('display.max_columns', 100)
plt.style.use('classic')

In [2]:
def new_fig(figsize=(8.9, 5)):
    plt.close()
    fig = plt.figure(figsize=figsize)
    ax = fig.add_subplot(111)
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(3)    
    ax.yaxis.grid(linestyle='--')
    for tick in ax.xaxis.get_major_ticks():
        tick.label.set_fontsize(18) 
    for tick in ax.yaxis.get_major_ticks():
        tick.label.set_fontsize(18) 
    for line in ax.get_xticklines() + ax.get_yticklines():
        line.set_markersize(8)
    return ax

In [ ]:
year = '2015'

out_path = '/lus/theta-fs0/projects/AMASE/rlewis/darshan/organized/' + year + '_summary.csv'
f = open(outpath, 'w+')

f.write(('jid,start_time,end_time,nprocs,cul_iotime,posix_time,mpiio_time,posix_cnt,mpiio_cnt,'
        'read_bytes,write_bytes,wr_bytes,unique_bytes,shared_bytes,read_nf,write_nf,wr_nf,unique_nf,'
        'shared_nf,posix_wr_time,posix_rd_time,posix_meta_time,mpiio_wr_time,mpiio_rd_time,mpiio_meta_time\n'))

for filename in glob.glob("/lus/theta-fs0/projects/AMASE/rlewis/darshan/organized/" + year + "/*.csv"):
    df = pd.read_csv(filename, header=0)
    
    job_id = df['jobid'][0]
    start_time = df['start_time'][0]
    end_time = df['end_time'][0]
    nprocs = df['nprocs'][0]
    read_bytes = df[(df['CP_BYTES_READ'] > 0) & (df['CP_BYTES_WRITTEN'] <= 0)]['CP_BYTES_READ'].sum()
    write_bytes = df[(df['CP_BYTES_WRITTEN'] > 0) & (df['CP_BYTES_READ'] <= 0)]['CP_BYTES_WRITTEN'].sum()
    wr_bytes = df[(df['CP_BYTES_WRITTEN'] > 0) & (df['CP_BYTES_READ'] > 0)]['CP_BYTES_WRITTEN'].sum() + df[(df['CP_BYTES_WRITTEN'] > 0) & (df['CP_BYTES_READ'] > 0)]['CP_BYTES_READ'].sum()
    unique_bytes = df[df['CP_INDEP_OPENS'] > 0]['CP_BYTES_READ'].sum() + df[df['CP_INDEP_OPENS'] > 0]['CP_BYTES_WRITTEN'].sum()
    shared_bytes = df[df['CP_COLL_OPENS'] > 0]['CP_BYTES_READ'].sum() + df[df['CP_COLL_OPENS'] > 0]['CP_BYTES_WRITTEN'].sum()
    posix_wr_time = df['CP_F_POSIX_WRITE_TIME'].sum()
    posix_rd_time = df['CP_F_POSIX_READ_TIME'].sum()
    posix_meta_time = df['CP_F_POSIX_META_TIME'].sum()
    mpiio_wr_time = df['CP_F_MPI_WRITE_TIME'].sum()
    mpiio_rd_time = df['CP_F_MPI_READ_TIME'].sum()
    mpiio_meta_time = df['CP_F_MPI_META_TIME'].sum()
    posix_time = posix_wr_time + posix_rd_time + posix_meta_time
    mpiio_time = mpiio_wr_time + mpiio_rd_time + mpiio_meta_time
    cul_iotime = posix_time + mpiio_time
    read_nf = df[df['CP_POSIX_READS'] > 0].count()[0]
    write_nf = df[df['CP_POSIX_WRITES'] > 0].count()[0]
    wr_nf = df[(df['CP_POSIX_READS'] > 0) & (df['CP_POSIX_WRITES'] > 0)].count()[0]
    unique_nf = df[df['CP_INDEP_OPENS'] > 0].count()[0]
    shared_nf = df[df['CP_COLL_OPENS'] > 0].count()[0]
    posix_cnt = read_nf + write_nf + wr_nf
    mpiio_cnt = unique_nf + shared_nf
    
    f.write('{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{},{}\n'.format(job_id,
            start_time,end_time,nprocs,cul_iotime,posix_time,mpiio_time,posix_cnt,mpiio_cnt,read_bytes,write_bytes,
            wr_bytes,unique_bytes,shared_bytes,read_nf,write_nf,wr_nf,unique_nf,shared_nf,posix_wr_time,posix_rd_time,
            posix_meta_time,mpiio_wr_time,mpiio_rd_time,mpiio_meta_time))

f.close()

In [57]:
# df['CP_F_POSIX_WRITE_TIME'].describe()
# df['CP_F_READ_END_TIMESTAMP'].describe()

count         98.0
mean     2368840.0
std            0.0
min      2368840.0
25%      2368840.0
50%      2368840.0
75%      2368840.0
max      2368840.0
Name: CP_F_READ_END_TIMESTAMP, dtype: float64